In [6]:
# chose the files that you want to use
import pickle
import json
import datetime
import timedelta
import os
import sp3
import matplotlib.pyplot as plt

RawDataFolder = 'sp3_test'
data_format = 'pickle'

# load the sp3 file
if data_format == 'sp3':
    product = sp3.Product.from_file(os.path.join(os.getcwd(), 'RawData', RawDataFolder, 'GFZOP_RSO_L64_G_20231225_220000_20231226_120000_v03.sp3'))

if data_format == 'pickle':
    # open the pickle file
    with open(os.path.join(os.getcwd(), 'RawData', RawDataFolder, 'L64_sp3_data.pkl'), 'rb') as file:
        product = pickle.load(file)

# Create some buffer to ensure a TLE capture the entire timeframe
start_time = product.satellites[0].records[0].time
end_time = product.satellites[0].records[-1].time
start_time_range = start_time - timedelta(days=2)
end_time_range = start_time + timedelta(days=2)
print('Start date: {}'.format(start_time))
print('End date: {}'.format(end_time))

with open(os.path.join(os.getcwd(), 'RawData', RawDataFolder, 'grace-fo-1-tle-43476.json'), "r") as file:
    tle_raw_l64 = json.load(file)

tles = [entry for entry in tle_raw_l64 if start_time_range <= datetime.fromisoformat(entry['EPOCH']).replace(tzinfo=tzutc()) <= end_time_range]

print("Number of tles available in the time range: ", len(tles))

## PLOT THE SPREAD OF TLEs BETWEEEN THE TIME RANGES
epoch_times = [datetime.fromisoformat(entry['EPOCH']) for entry in tle_raw_l64]
time_diffs = [1 for _ in epoch_times] # in hours

# Plot
plt.figure(figsize=(12, 6))
plt.plot(epoch_times, time_diffs, marker='o', linestyle='-', color='blue')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gcf().autofmt_xdate()  # for slanting date labels
plt.title("TLE's Available For Start and End Time")
plt.xlabel("UTC Time")
plt.ylabel("Time Difference (hours)")
plt.axvline(start_time, color='red', linestyle='--', label='Start Time')
plt.axvline(end_time, color='red', linestyle='--', label='End Time')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.xticks(rotation=45)
plt.xlim(start_time_range, end_time_range)
plt.show()

ModuleNotFoundError: No module named 'timedelta'

2804

In [1]:
from typing import List, Tuple, Union
import numpy as np
from pyproj import Transformer
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from astropy.coordinates import CartesianRepresentation, CartesianDifferential, GCRS, ITRS
from astropy.time import Time
import astropy.units as u

import datetime
from typing import Union

def utc_jd_date(day: int, month: int, year: int, hours: int, minutes: int, seconds: int, mjd: bool = False, midnight: bool = False) -> Union[float, int]:
    if midnight:
        # Convert to datetime object (without hours and seconds)
        date = datetime.datetime(year, month, day)
    else:
        # Convert to datetime object (with hours and seconds)
        date = datetime.datetime(year, month, day, hours, minutes, seconds)

    # Calculate MJD
    mjd_value = (date - datetime.datetime(1858, 11, 17)).total_seconds() / 86400.0
    if mjd:
        return mjd_value
    else:
        # Convert to JD
        jd = mjd_value + 2400000.5
        return jd
    
def eci2ecef_astropy(eci_pos: np.ndarray, eci_vel: np.ndarray, mjd: float) -> Tuple[np.ndarray, np.ndarray]:
    """
    Convert ECI (Earth-Centered Inertial) coordinates to ECEF (Earth-Centered, Earth-Fixed) coordinates using Astropy.

    Parameters
    ----------
    eci_pos : np.ndarray
        ECI position vectors.
    eci_vel : np.ndarray
        ECI velocity vectors.
    mjd : float
        Modified Julian Date.

    Returns
    -------
    tuple
        ECEF position vectors and ECEF velocity vectors.
    """
    # Convert MJD to isot format for Astropy
    time_utc = Time(mjd, format="mjd", scale='utc')

    # Convert ECI position and velocity to ECEF coordinates using Astropy
    eci_cartesian = CartesianRepresentation(eci_pos.T * u.km)
    eci_velocity = CartesianDifferential(eci_vel.T * u.km / u.s)
    gcrs_coords = GCRS(eci_cartesian.with_differentials(eci_velocity), obstime=time_utc)
    itrs_coords = gcrs_coords.transform_to(ITRS(obstime=time_utc))

    # Get ECEF position and velocity from Astropy coordinates
    ecef_pos = np.column_stack((itrs_coords.x.value, itrs_coords.y.value, itrs_coords.z.value))
    ecef_vel = np.column_stack((itrs_coords.v_x.value, itrs_coords.v_y.value, itrs_coords.v_z.value))

    return ecef_pos, ecef_vel

# Instantiate the Transformer for ECEF to LLA conversion
transformer = Transformer.from_crs(
    "EPSG:4978", # WGS-84 (ECEF)
    "EPSG:4326", # WGS-84 (LLA)
    always_xy=True
)

time, H_diffs, C_diffs, L_diffs, positions, velocity = grace1_ephemeris[0], grace1_ephemeris[1], grace1_ephemeris[2], grace1_ephemeris[3], grace1_ephemeris[4], grace1_ephemeris[6]


# Initialize lists for latitudes and longitudes
lats, lons = [], []

# Convert ECI positions to ECEF, then to latitude and longitude
for i, position in enumerate(positions):
    # Convert the time to MJD format
    mjd = utc_jd_date(time[i].day, time[i].month, time[i].year, time[i].hour, time[i].minute, time[i].second, mjd=True)

    # Assuming zero velocity for the conversion, update if you have velocity data
    ecef_pos, _ = eci2ecef_astropy(np.array([position]), np.array([velocity[i]]), mjd)
    
    # Ensure ecef_pos is a single vector of length 3
    ecef_pos = ecef_pos.flatten()  # Flatten the array to 1D if it's not already

    # Transform ECEF to lat/lon
    lat, lon, _ = transformer.transform(ecef_pos[0], ecef_pos[1], ecef_pos[2])
    lats.append(lat)
    lons.append(lon)

# Prepare the Basemap for each dataset
fig, axes = plt.subplots(3, 1, figsize=(15, 30))

# Dataset names for titles and legends
datasets = ['H_diffs', 'C_diffs', 'L_diffs']

# Plotting H_diffs, C_diffs, and L_diffs on separate maps
for i, (diffs, dataset_name) in enumerate(zip([H_diffs, C_diffs, L_diffs], datasets)):
    m = Basemap(projection='mill', llcrnrlat=-90, urcrnrlat=90, llcrnrlon=-180, urcrnrlon=180, resolution='c', ax=axes[i])
    m.drawcoastlines()
    m.drawcountries()
    m.drawmapboundary(fill_color='xkcd:light grey')

    # Scatter plot
    xpt, ypt = m(lats, lons)
    scatter_plot = m.scatter(xpt, ypt, c=diffs, alpha=0.5, s=10)

    # Adding title and legend
    axes[i].set_title(f'Map for {dataset_name}', fontsize=14)
    plt.colorbar(scatter_plot, ax=axes[i], orientation='vertical', shrink=0.5, label='Value')

plt.tight_layout()
plt.show()

NameError: name 'grace1_ephemeris' is not defined